In [1]:
# ================================================================
# 1 — Imports & Config (linked to Exploration Notebook outputs)
# ================================================================
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import random

# Reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Paths
ROOT = Path(".").resolve()
DATA_DIR = ROOT / "data"
PROCESSED_DIR = DATA_DIR / "processed"
PROCESSED_DIR.mkdir(exist_ok=True, parents=True)

# Load selected stations from exploration notebook output
REPORTS_DIR = ROOT / "reports"
selection_file = REPORTS_DIR / "selected_buoys.json"
if not selection_file.exists():
    raise FileNotFoundError(
        f"Selection file not found at {selection_file}. "
        "Run the exploration notebook first."
    )

with open(selection_file, "r") as f:
    selection = json.load(f)

MAIN_STATION = selection["main_buoy"]
EXTRA_TRAIN_STATIONS = selection.get("extra_train_buoys", [])
GEN_STATION = selection["generalization_buoy"]

# Domain config
SEQ_LEN = 72
HORIZON = 6
STRIDE = 3
MAX_INTERP_GAP = 12
K_DURATION = 2
THRESH_MODE = "seasonal_q90"  # keep in physical units (meters)

# Columns
SENSOR_COLS = [
    'AtmosphericPressure', 'WindDirection', 'WindSpeed', 'Gust',
    'WaveHeight', 'WavePeriod', 'MeanWaveDirection', 'Hmax',
    'AirTemperature', 'DewPoint', 'SeaTemperature', 'RelativeHumidity'
]
TIME_FEATS = ['HOUR_sin', 'HOUR_cos', 'WEEKDAY_sin', 'WEEKDAY_cos']
PLACEHOLDERS = {"NaN", "-999", -999, -999.0}


# Consistency for label unit fix later
# keep a raw copy of WaveHeight for threshold calculations.
# Scaling done after train/val/test split to prevent leakage.
RAW_WAVEHEIGHT_COL = 'WaveHeight_raw'

print("=== Config ===")
print(f"Main: {MAIN_STATION}, Extra: {EXTRA_TRAIN_STATIONS}, Gen: {GEN_STATION}")
print(f"SEQ_LEN={SEQ_LEN}, HORIZON={HORIZON}, STRIDE={STRIDE}")


C:\Users\pesic\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


=== Config ===
Main: M2, Extra: ['M3'], Gen: M5
SEQ_LEN=72, HORIZON=6, STRIDE=3


In [2]:
# ========================================
# 2 — Load & Filter (main + extras + gen) 
# ========================================
RAW_CSV = DATA_DIR / "Buoy_raw.csv"
if not RAW_CSV.exists():
    raise FileNotFoundError(f"Dataset not found at {RAW_CSV}")

# Load dataset
df_raw = pd.read_csv(RAW_CSV, low_memory=False)

# Drop the units row if present
df_raw = df_raw[df_raw['station_id'] != 'station_id']
df_raw = df_raw[df_raw['station_id'].notna()]

# Ensure numeric for all sensor cols
for col in SENSOR_COLS:
    if col in df_raw.columns:
        df_raw[col] = pd.to_numeric(df_raw[col], errors='coerce')

# Convert time to datetime
df_raw['time'] = pd.to_datetime(df_raw['time'], errors='coerce', utc=True)

# Replace placeholders with NaN
for col in SENSOR_COLS:
    if col in df_raw.columns:
        df_raw[col] = df_raw[col].replace(list(PLACEHOLDERS), np.nan)

# Preserve raw WaveHeight for label computation
RAW_WAVEHEIGHT_COL = "WaveHeight_raw"
if 'WaveHeight' not in df_raw.columns:
    raise KeyError("WaveHeight column not found in dataset.")
df_raw[RAW_WAVEHEIGHT_COL] = df_raw['WaveHeight'].copy()

# Remove location leakage — drop lon/lat entirely
if 'longitude' in df_raw.columns:
    df_raw.drop(columns=['longitude'], inplace=True)
if 'latitude' in df_raw.columns:
    df_raw.drop(columns=['latitude'], inplace=True)

# Encode cyclic wind direction features (replaces raw angles)
for dir_col in ['WindDirection', 'MeanWaveDirection']:
    if dir_col in df_raw.columns:
        radians = np.deg2rad(df_raw[dir_col] % 360)
        df_raw[f"{dir_col}_sin"] = np.sin(radians)
        df_raw[f"{dir_col}_cos"] = np.cos(radians)
        df_raw.drop(columns=[dir_col], inplace=True)

# Update SENSOR_COLS
SENSOR_COLS = [
    c for c in SENSOR_COLS + ['WindDirection_sin', 'WindDirection_cos',
                              'MeanWaveDirection_sin', 'MeanWaveDirection_cos']
    if c in df_raw.columns and c not in ['WindDirection', 'MeanWaveDirection']
]

# Filter main + extra stations for training
train_ids = [MAIN_STATION] + EXTRA_TRAIN_STATIONS
df_train_raw = df_raw[df_raw['station_id'].isin(train_ids)].copy()

# Filter generalization station separately
df_gen_raw = df_raw[df_raw['station_id'] == GEN_STATION].copy()

print(f"Loaded TRAIN domain ({train_ids}): {len(df_train_raw)} rows "
      f"from {df_train_raw['time'].min()} to {df_train_raw['time'].max()}")
print(f"Loaded GEN buoy ({GEN_STATION}): {len(df_gen_raw)} rows "
      f"from {df_gen_raw['time'].min()} to {df_gen_raw['time'].max()}")


Loaded TRAIN domain (['M2', 'M3']): 231519 rows from 2001-05-03 14:00:00+00:00 to 2017-11-28 10:00:00+00:00
Loaded GEN buoy (M5): 101137 rows from 2004-10-18 10:00:00+00:00 to 2017-11-28 10:00:00+00:00


In [3]:
# ================================================================
# 3 — Clean (preserve raw WH), compute masks/deltas before fill,
#           relative WH in meters, thresholds in meters.
# ================================================================
def compute_thresholds(df, mode="seasonal_q90", raw_col=RAW_WAVEHEIGHT_COL):
    """Compute monthly or global 90th percentile thresholds from raw WH (meters)."""
    if raw_col not in df.columns:
        raise KeyError(f"{raw_col} not found for threshold computation.")
    if mode == "seasonal_q90":
        return df.groupby(df.index.month)[raw_col].quantile(0.90)
    elif mode == "global_q90":
        thr = df[raw_col].quantile(0.90)
        return pd.Series([thr] * 12, index=range(1, 13))
    else:
        raise ValueError(f"Unknown threshold mode: {mode}")

def clean_and_engineer(df, sensor_cols=SENSOR_COLS):
    # Sort and drop duplicate timestamps
    df = df.sort_values('time').drop_duplicates('time')

    # Keep metadata
    meta_cols = [c for c in ['station_id'] if c in df.columns]  # lon/lat removed
    meta_data = df[meta_cols].iloc[0]

    # Reindex to full hourly range
    full_idx = pd.date_range(df['time'].min(), df['time'].max(), freq='H', tz='UTC')
    df = df.set_index('time').reindex(full_idx)

    # Restore metadata
    for col in meta_cols:
        df[col] = meta_data[col]

    # Masks & deltas BEFORE filling
    
    masks = df[sensor_cols].isna().astype(np.int8)
    deltas = pd.DataFrame(index=df.index)
    for col in sensor_cols:
        count = 0
        delta_col = []
        for missing in masks[col]:
            count = 0 if missing == 0 else count + 1
            delta_col.append(count)
        deltas[col] = delta_col
    masks.columns = [f"{c}_mask" for c in masks.columns]
    deltas.columns = [f"{c}_delta" for c in deltas.columns]

    
    # Interpolation & seasonal median fill
    
    for col in sensor_cols:
        df[col] = df[col].interpolate(method='time', limit=MAX_INTERP_GAP, limit_direction='both')
        month_median = df[col].groupby(df.index.month).transform('median')
        df[col] = df[col].fillna(month_median)

    
    # Compute thresholds in meters from WaveHeight_raw
    
    thresholds = compute_thresholds(df, mode=THRESH_MODE, raw_col=RAW_WAVEHEIGHT_COL)

    # Relative WH in meters (unscaled for now)
    month_series = df.index.month
    df["WaveHeight_rel"] = (df[RAW_WAVEHEIGHT_COL] - month_series.map(thresholds)) / month_series.map(thresholds)
    df["WaveHeight_rel"] = df["WaveHeight_rel"].fillna(0)

    # Replace any residual NaNs in sensor cols (before scaling)
    for col in sensor_cols:
        df[col] = df[col].fillna(0)

    
    # Time features
    df['HOUR_sin'] = np.sin(2 * np.pi * df.index.hour / 24)
    df['HOUR_cos'] = np.cos(2 * np.pi * df.index.hour / 24)
    df['WEEKDAY_sin'] = np.sin(2 * np.pi * df.index.weekday / 7)
    df['WEEKDAY_cos'] = np.cos(2 * np.pi * df.index.weekday / 7)

    
    # Attach masks & deltas
    
    df = pd.concat([df, masks, deltas], axis=1)

    return df, thresholds


In [4]:
# ================================================================
#  4 — Clean all training buoys & generalization buoy
#   Preserve WaveHeight_raw and clean NaNs/Inf
# ================================================================

def save_clean_npz(df, station_id, thresholds):
    """Save cleaned dataframe + thresholds for one buoy."""
    # Ensure no column ordering issues
    df = df.copy()
    if RAW_WAVEHEIGHT_COL not in df.columns:
        raise KeyError(f"{RAW_WAVEHEIGHT_COL} missing in cleaned dataframe for {station_id}")

    # Final NaN/Inf check — keep but fill with 0 for safe storage
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.fillna(0)

    path = PROCESSED_DIR / f"clean_{station_id}.npz"
    np.savez_compressed(
        path,
        data=df.to_numpy(),
        index=df.index.astype(str).to_numpy(),
        columns=df.columns.to_numpy(),
        thresholds=thresholds.to_numpy(),
        threshold_months=thresholds.index.to_numpy()
    )
    print(f"Saved cleaned {station_id} → {path} ({len(df)} hours, cols={len(df.columns)})")

def load_clean_npz(path):
    """Load cleaned dataframe + thresholds from disk."""
    npz = np.load(path, allow_pickle=True)
    df = pd.DataFrame(
        data=npz["data"],
        index=pd.to_datetime(npz["index"], utc=True),
        columns=npz["columns"]
    )
    thresholds = pd.Series(npz["thresholds"], index=npz["threshold_months"])
    return df, thresholds

# Clean and save each training buoy separately

df_train_clean_list = []
for sid in train_ids:
    df_buoy_clean, thr_buoy = clean_and_engineer(
        df_train_raw[df_train_raw['station_id'] == sid]
    )
    df_buoy_clean.index = pd.to_datetime(df_buoy_clean.index, utc=True)
    save_clean_npz(df_buoy_clean, sid, thr_buoy)
    df_train_clean_list.append(df_buoy_clean)

# Merge cleaned training buoys (for diagnostics only)
df_train_clean = pd.concat(df_train_clean_list).sort_index()

# Clean & save generalization buoy

df_gen_clean, thr_gen = clean_and_engineer(df_gen_raw)
df_gen_clean.index = pd.to_datetime(df_gen_clean.index, utc=True)
save_clean_npz(df_gen_clean, GEN_STATION, thr_gen)


Saved cleaned M2 → C:\Users\pesic\Desktop\GRU\data\processed\clean_M2.npz (145269 hours, cols=50)
Saved cleaned M3 → C:\Users\pesic\Desktop\GRU\data\processed\clean_M3.npz (130958 hours, cols=50)
Saved cleaned M5 → C:\Users\pesic\Desktop\GRU\data\processed\clean_M5.npz (114937 hours, cols=50)


In [5]:
# ================================================================
# 5 — Sequence generation
# ================================================================
def make_sequences(df_b, thresholds, seq_len=SEQ_LEN, horizon=HORIZON, stride=STRIDE):
    """Create sliding window sequences with danger labels in meters."""
    mask_cols = [c for c in df_b.columns if c.endswith('_mask')]
    delta_cols = [c for c in df_b.columns if c.endswith('_delta')]
    feature_cols = SENSOR_COLS + ['WaveHeight_rel'] + mask_cols + delta_cols + TIME_FEATS

    X_seq, y_cls, y_reg, dates_seq = [], [], [], []

    for i in range(0, len(df_b) - seq_len - horizon, stride):
        X_window = df_b.iloc[i:i+seq_len][feature_cols].values

        # Use raw WH (meters) for labels
        future_wvht_raw = df_b.iloc[i+seq_len:i+seq_len+horizon][RAW_WAVEHEIGHT_COL].values
        month = df_b.index[i+seq_len].month
        thresh = thresholds.loc[month]
        if np.isnan(thresh):
            continue

        danger = int(any(
            np.convolve((future_wvht_raw > thresh).astype(int),
                        np.ones(K_DURATION, dtype=int),
                        'valid') >= K_DURATION
        ))

        y_cls.append(danger)
        y_reg.append(future_wvht_raw.mean())  # regression target in meters
        X_seq.append(X_window)
        dates_seq.append(df_b.index[i+seq_len])

    return (
        np.array(X_seq, dtype=np.float32),
        np.array(y_cls, dtype=np.int64),
        np.array(y_reg, dtype=np.float32),
        pd.DatetimeIndex(dates_seq)
    )

# Generate sequences for each buoy in training domain
X_train_all, y_train_cls_all, y_train_reg_all, dates_train_all = [], [], [], []
for sid in train_ids:
    df_b, thr_b = load_clean_npz(PROCESSED_DIR / f"clean_{sid}.npz")
    X_seq, y_cls, y_reg, dates_seq = make_sequences(df_b, thr_b)
    X_train_all.append(X_seq)
    y_train_cls_all.append(y_cls)
    y_train_reg_all.append(y_reg)
    dates_train_all.append(dates_seq)

# Merge all training sequences
X_main = np.vstack(X_train_all)
y_main_cls = np.hstack(y_train_cls_all)
y_main_reg = np.hstack(y_train_reg_all)
dates_main = pd.DatetimeIndex(np.hstack(dates_train_all))

# Generalization buoy

df_gen_b, thr_gen = load_clean_npz(PROCESSED_DIR / f"clean_{GEN_STATION}.npz")
X_gen, y_gen_cls, y_gen_reg, dates_gen = make_sequences(df_gen_b, thr_gen)

print(f"Train domain sequences: {X_main.shape}, Danger%={100*y_main_cls.mean():.2f}")
print(f"Gen sequences: {X_gen.shape}, Danger%={100*y_gen_cls.mean():.2f}")


Train domain sequences: (92024, 72, 47), Danger%=9.22
Gen sequences: (38287, 72, 47), Danger%=8.05


In [6]:
# ================================================================
# 6 — Chronological split with strict non-overlapping boundaries
# ================================================================
from sklearn.preprocessing import StandardScaler
import numpy as np

# 1. Chronological split by percentage
# -----------------
p = pd.Series(dates_main).rank(pct=True).values
train_mask = p < 0.70
val_mask   = (p >= 0.70) & (p < 0.85)
test_mask  = p >= 0.85

X_train, y_train_cls, y_train_reg, dates_train = (
    X_main[train_mask], y_main_cls[train_mask], y_main_reg[train_mask], dates_main[train_mask]
)
X_val, y_val_cls, y_val_reg, dates_val = (
    X_main[val_mask], y_main_cls[val_mask], y_main_reg[val_mask], dates_main[val_mask]
)
X_test, y_test_cls, y_test_reg, dates_test = (
    X_main[test_mask], y_main_cls[test_mask], y_main_reg[test_mask], dates_main[test_mask]
)

# 2. Station-aware overlap check
# -----------------
def remove_overlap(dates_a, dates_b, X_a, y_a_cls, y_a_reg, stations_a=None, stations_b=None):
    if stations_a is not None and stations_b is not None:
        tuples_a = list(zip(stations_a, dates_a))
        tuples_b = set(zip(stations_b, dates_b))
        overlap_mask = [t in tuples_b for t in tuples_a]
    else:
        overlap_mask = pd.Series(dates_a).isin(pd.Series(dates_b))
    if any(overlap_mask):
        print(f"Removing {sum(overlap_mask)} overlapping samples from split.")
        keep_mask = ~pd.Series(overlap_mask)
        return dates_a[keep_mask], X_a[keep_mask], y_a_cls[keep_mask], y_a_reg[keep_mask]
    return dates_a, X_a, y_a_cls, y_a_reg

dates_val, X_val, y_val_cls, y_val_reg = remove_overlap(dates_val, dates_test, X_val, y_val_cls, y_val_reg)


# 3. Final overlap check
# -----------------
train_val_ov  = pd.Series(dates_train).isin(dates_val).sum()
val_test_ov   = pd.Series(dates_val).isin(dates_test).sum()
train_test_ov = pd.Series(dates_train).isin(dates_test).sum()

if any(x > 0 for x in [train_val_ov, val_test_ov, train_test_ov]):
    raise ValueError(f"Overlap remains after cleaning: Train–Val={train_val_ov}, Val–Test={val_test_ov}, Train–Test={train_test_ov}")
else:
    print("No overlaps between Train, Val, and Test splits.")

# 4. Alignment + NaN/Inf cleaning for each split (dates included)
# -----------------
def align_and_clean(X, y_cls, y_reg, dates, split_name):
    # Align by min length
    min_len = min(len(X), len(y_cls), len(y_reg), len(dates))
    if not (len(X) == len(y_cls) == len(y_reg) == len(dates)):
        print(f"{split_name}: Mismatch in lengths -> trimming to {min_len} samples")
        X, y_cls, y_reg, dates = X[:min_len], y_cls[:min_len], y_reg[:min_len], dates[:min_len]
    # Clean NaN/Inf
    mask = (
        np.isfinite(X).all(axis=(1, 2)) &
        np.isfinite(y_cls) &
        np.isfinite(y_reg)
    )
    removed = np.count_nonzero(~mask)
    kept = np.count_nonzero(mask)
    if removed > 0:
        print(f"{split_name}: Removed {removed} / {len(mask)} samples (kept {kept}) due to non-finite values")
    else:
        print(f"{split_name}: All samples finite ({kept} total)")
    return X[mask], y_cls[mask], y_reg[mask], dates[mask]

X_train, y_train_cls, y_train_reg, dates_train = align_and_clean(X_train, y_train_cls, y_train_reg, dates_train, "Train")
X_val,   y_val_cls,   y_val_reg,   dates_val   = align_and_clean(X_val,   y_val_cls,   y_val_reg,   dates_val,   "Val")
X_test,  y_test_cls,  y_test_reg,  dates_test  = align_and_clean(X_test,  y_test_cls,  y_test_reg,  dates_test,  "Test")
X_gen,   y_gen_cls,   y_gen_reg,   dates_gen   = align_and_clean(X_gen,   y_gen_cls,   y_gen_reg,   dates_gen,   "Gen")

# 5. Train-only scaling
# -----------------
num_features = X_train.shape[2]
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train.reshape(-1, num_features)).reshape(X_train.shape)
X_val_scaled   = scaler.transform(X_val.reshape(-1, num_features)).reshape(X_val.shape)
X_test_scaled  = scaler.transform(X_test.reshape(-1, num_features)).reshape(X_test.shape)
X_gen_scaled   = scaler.transform(X_gen.reshape(-1, num_features)).reshape(X_gen.shape)

# 6. Final verification
# -----------------
for name, arr in [
    ("X_train", X_train_scaled), ("y_train_cls", y_train_cls), ("y_train_reg", y_train_reg),
    ("X_val", X_val_scaled),     ("y_val_cls", y_val_cls),     ("y_val_reg", y_val_reg),
    ("X_test", X_test_scaled),   ("y_test_cls", y_test_cls),   ("y_test_reg", y_test_reg),
    ("X_gen", X_gen_scaled),     ("y_gen_cls", y_gen_cls),     ("y_gen_reg", y_gen_reg)
]:
    if not np.isfinite(arr).all():
        raise ValueError(f"{name} still contains NaN or Inf values!")

print("All features and targets are finite after cleaning.")

# 7. Save to disk
# -----------------
np.savez_compressed(
    PROCESSED_DIR / f"model_ready_{MAIN_STATION}.npz",
    X_train=X_train_scaled, y_train_cls=y_train_cls, y_train_reg=y_train_reg, dates_train=dates_train,
    X_val=X_val_scaled,     y_val_cls=y_val_cls,     y_val_reg=y_val_reg,     dates_val=dates_val,
    X_test=X_test_scaled,   y_test_cls=y_test_cls,   y_test_reg=y_test_reg,   dates_test=dates_test
)
np.savez_compressed(
    PROCESSED_DIR / f"model_ready_{GEN_STATION}.npz",
    X_gen=X_gen_scaled,     y_gen_cls=y_gen_cls,     y_gen_reg=y_gen_reg,     dates_gen=dates_gen
)


No overlaps between Train, Val, and Test splits.
Train: All samples finite (64416 total)
Val: All samples finite (13804 total)
Test: All samples finite (13804 total)
Gen: All samples finite (38287 total)
All features and targets are finite after cleaning.
